In [3]:
DATA_ROOT = 'data/'

In [2]:
# DAY01
nums = [int(x) for x in open(DATA_ROOT + '01.txt', 'r').readlines()]

reg = {}

TOTAL = 2020
for i, n in enumerate(nums):
    rest = TOTAL - n
    if rest in reg:
        print(f'{n} * {rest} = {n * rest}')
    reg[n] = i
    
for i, n1 in enumerate(nums):
    for j in range(i + 1, len(nums)):
        n2 = nums[j]
        rest = TOTAL - n1 - n2
        if rest in reg and reg[rest] > j:
            print(f'{n1} * {n2} * {rest} = {n1 * n2 * rest}')

1564 * 456 = 713184
764 * 857 * 399 = 261244452


In [3]:
# DAY02
def parse_line(line):
    parts = line.strip().split()
    counts = list(map(int, parts[0].split('-')))
    return (counts[0], counts[1], parts[1][0], parts[2])

def is_valid1(desc):
    cnt1, cnt2, ch, pwd = desc
    cnt = pwd.count(ch)
    return cnt1 <= cnt <= cnt2

def is_valid2(desc):
    cnt1, cnt2, ch, pwd = desc
    return (pwd[cnt1 - 1] == ch) ^ (pwd[cnt2 - 1] == ch)
    
passwords = list(map(parse_line, open(DATA_ROOT + '02.txt', 'r').readlines()))
res1 = sum([is_valid1(pwd) for pwd in passwords])
res2 = sum([is_valid2(pwd) for pwd in passwords])

print(f'Number of valid passwords: {res1}, {res2}')

Number of valid passwords: 548, 502


In [7]:
# DAY03
import math 

def count_trees(rows, dx, dy):
    h = len(rows)
    if h == 0:
        return 0
    w = len(rows[0])
    x, y = 0, 0
    res = 0
    while y < h:
        res += rows[y][x % w] == '#'
        x += dx
        y += dy
    return res

rows = [x.strip() for x in open(DATA_ROOT + '03.txt', 'r').readlines()]
res1 = count_trees(rows, 3, 1)
print(f'Num trees 1: {res1}')

SLOPES = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]
res2 = math.prod((count_trees(rows, x, y) for (x, y) in SLOPES))
print(f'Num trees 2: {res2}')

Num trees 1: 242
Num trees 2: 2265549792


In [85]:
# DAY04
def read_doc(line):
    return dict([x.split(':') for x in line.split()])

REQUIRED_FIELDS = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']

def is_valid1(passport):
    return all(f in passport for f in REQUIRED_FIELDS)

def in_range(int_str, minv, maxv):
    try:
        return minv <= int(int_str) <= maxv
    except:
        return False

def is_valid_height(height_str):
    match = re.match(r"^(?P<in>\d+)in|(?P<cm>\d+)cm$", height_str)
    if match == None:
        return False
    hcm, hin = match.group('cm'), match.group('in')
    if hcm != None:
        return in_range(hcm, 150, 193)
    elif hin != None:
        return in_range(hin, 59, 76)
    else:
        return False

VALID_EYE_COLORS = ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']

def is_valid2(passport):
    return is_valid1(passport) \
        and in_range(passport['byr'], 1920, 2002) \
        and in_range(passport['iyr'], 2010, 2020) \
        and in_range(passport['eyr'], 2020, 2030) \
        and passport['ecl'] in VALID_EYE_COLORS \
        and re.match(r"^(\d{9})$", passport['pid']) is not None \
        and re.match(r"^#[0-9a-f]{6}$", passport['hcl']) is not None \
        and is_valid_height(passport['hgt'])
    
lines = open(DATA_ROOT + '04.txt', 'r').read()
passports = [read_doc(line) for line in lines.split("\n\n")]

print(f'Valid passports 1: {sum(map(is_valid1, passports))}')
print(f'Valid passports 2: {sum(map(is_valid2, passports))}')

Valid passports 1: 264
Valid passports 2: 224


In [4]:
# DAY05
from functools import reduce

def get_id(seat):
    return reduce(lambda res, c: res * 2 + (c in 'BR'), seat, 0)

lines = [x.strip() for x in open(DATA_ROOT + '05.txt', 'r').readlines()]
ids = [get_id(line) for line in lines]

idset = set(ids)
seats = [id for id in range(1 << len(lines[0]))
    if id not in idset and id + 1 in idset and id - 1 in idset]

print(f'Max ID: {max(ids)}, Seat: {seats[0]}')

Max ID: 801, Seat: 597


In [32]:
# DAY06
groups = [g.split() for g in open(DATA_ROOT + '06.txt', 'r').read().split('\n\n')]

res1, res2 = 0, 0
for group in groups:
    counts = {}
    for answers in group:
        for ans in answers:
            if ans in counts:
                counts[ans] += 1
            else: 
                counts[ans] = 1
    res1 += len(counts)
    res2 += sum(counts[k] == len(group) for k in counts.keys())

print(f'Answer 1: {res1}, Answer 2: {res2}')

Answer 1: 6310, Answer 2: 3193


In [11]:
# DAY07
def parse_mapping(line):
    parts = line.split()
    bag_color = ' '.join(parts[0:2])
    res = {}
    for i in range(4, len(parts), 4):
        if parts[i] != 'no':
            res[' '.join(parts[i + 1:i + 3])] = int(parts[i])
    return (bag_color, res)

def build_back_mapping(mapping):
    res = {}
    for (bag, children) in mapping.items():
        for child in children.keys():
            if child in res:
                res[child].add(bag)
            else:
                res[child] = set([bag])
    return res

def count_reachable(root, back_mapping):
    visited = set()
    def rec(node):
        if node in visited:
            return 0
        res = 1
        if node in back_mapping:
            for child in back_mapping[node]:
                res += rec(child)
        visited.add(node)
        return res
    return rec(root)
    
def count_contains(root, mapping):
    res = 1
    for (child, count) in mapping[root].items():
        res += count * count_contains(child, mapping)
    return res
    
ROOT = "shiny gold"

lines = [g.strip() for g in open(DATA_ROOT + '07.txt', 'r').readlines()]
mapping = dict(parse_mapping(line) for line in lines)
back_mapping = build_back_mapping(mapping)

print(f'Reachable: {count_reachable(ROOT, back_mapping) - 1}')
print(f'Contains: {count_contains(ROOT, mapping) - 1}')

Reachable: 370
Contains: 29547


In [4]:
# DAY08
import enum

class Op(enum.Enum): 
    acc = 1
    nop = 2
    jmp = 3
    
def parse_op(line):
    parts = line.split()
    return (Op[parts[0]], int(parts[1]))

def eval_ops(ops, acc=0):
    n = len(ops)
    visited = [False]*n
    ip = 0
    while ip >= 0 and ip < n:
        if visited[ip]:
            return (False, acc)
        visited[ip] = True

        (op, val) = ops[ip]
        if op == Op.jmp:
            ip += val - 1
        elif op == Op.acc:
            acc += val
        ip += 1
    return (True, acc)

def try_mutate_program(ops):
    for i, (op, val) in enumerate(ops):
        prevOp = op
        if op == Op.jmp:
            op = Op.nop
        elif op == Op.nop:
            op = Op.jmp
        else:
            continue
            
        ops[i] = (op, val)
        (terminated, acc) = eval_ops(ops)
        if terminated:
            return acc
        ops[i] = (prevOp, val)
    
ops = [parse_op(g.strip()) for g in open(DATA_ROOT + '08.txt', 'r').readlines()]

print(f'Answer 1: {eval_ops(ops)[1]}, Answer 2: {try_mutate_program(ops)}')

Answer 1: 1782, Answer 2: 797


In [36]:
# DAY09
import bisect

def get_first_non_sum(nums, k):
    window = sorted(nums[:k])
    for i in range(k, len(nums)):
        l, r = 0, k - 1
        s = nums[i]
        while l < r:
            if window[l] + window[r] < s:
                l += 1
            elif window[l] + window[r] > s:
                r -= 1
            else: 
                break
        if l == r:
            return s
        bisect.insort(window, s)
        del window[bisect.bisect_left(window, nums[i - k])]
        
def find_sum_range(nums, s):
    l, r = 0, 0
    cur_sum = nums[0]
    n = len(nums)
    while r < n - 1 and l < n - 1:
        if cur_sum < s:
            r += 1
            cur_sum += nums[r]
        elif cur_sum > s:
            cur_sum -= nums[l]
            l += 1
        else:
            break
    if l != r:
        return (l, r)
        
nums = [int(g) for g in open(DATA_ROOT + '09.txt', 'r').readlines()]

WINDOW_SIZE = 25

non_sum = get_first_non_sum(nums, WINDOW_SIZE)
print(f'First non-sum: {non_sum}')

(b, e) = find_sum_range(nums, non_sum)
print(f'Sum range: {min(nums[b:e + 1]) + max(nums[b:e + 1])}')

First non-sum: 85848519
Sum range: 13414198
